In [1]:
import numpy as np
import pandas as pd
import os, sys

import json

import plotly.graph_objects as go

In [67]:
data_dir = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/random/"

data_files = [file for file in os.listdir(data_dir) if ".csv" in file and "~lock" not in file]

total_dataframe = pd.DataFrame()
for datafile in data_files:
    tmp_df = pd.read_csv(os.path.join(data_dir, datafile), sep=";")
    total_dataframe = pd.concat((total_dataframe, tmp_df))

In [68]:
def jsonify(string: str):
    if type(string) != str:
        return ""
    tmp_str = string.replace("'", '"')
    tmp_str = tmp_str.replace("None", "null")
    tmp_str = tmp_str.replace("True", "true")
    tmp_str = tmp_str.replace("False", "false")

    return tmp_str

In [69]:


#print(total_dataframe.head())


pos_rowslist_dict = {"bns_maurice_1": [], "bns_maurice_2": [], "bns_maurice_3": [], "bns_maurice_4": []}

rowslist = []
for k, row in total_dataframe.iterrows():
    dec_str = jsonify(row["rise_decline"])
    #print(tmp_str)
    dec_dict = json.loads(dec_str)
    #print(dec_dict)

    rowslist.append(dec_dict)

    #print(row["chisquares"])
    pos_str = jsonify(row["chisquares"])
    if pos_str != "":
        pos_dict = json.loads(pos_str)
        #print(pos_dict)
        for key, value in pos_dict.items():
            value["ndet"] = dec_dict["ndet"]
            value["latest_det"] = dec_dict["jd_last"]
            value["first_det"] = dec_dict["jd_det"]
            value["kilonovaness"] = row["kilonovaness"]
            value["map_seed"] = row["map_seed"]
            value["map_dist"] = row["map_dist"]
            value["map_area"] = row["map_area"]
            if value["ndof"] > 0:
                value["red_chisqu"] = value["chisq"] / value["ndof"]
            else:
                value["red_chisqu"] = -999
            pos_rowslist_dict[key].append(value)
    #print(row["rise_decline"]["ndet"])


pos_df_dict = {}
for key in pos_rowslist_dict:
    pos_df_dict[key] = pd.DataFrame(pos_rowslist_dict[key]).sort_values(by="ndet")


In [75]:
keys = list(pos_df_dict.keys())
key = keys[0]

fig = go.Figure()

max_chisq = 50
n_bins = 15
bins = np.linspace(0, max_chisq, n_bins)

print(len(pos_df_dict[key]))
for key in keys[:]:
    tmp_data = pos_df_dict[key].copy()[np.logical_and(pos_df_dict[key]["chisq"] <= max_chisq, pos_df_dict[key]["chisq"] >= 0.8) ]

    tmp_hist = np.histogram(tmp_data["chisq"], bins=bins)
    #print(tmp_hist)

    fig.add_trace(go.Bar(x=bins, y = tmp_hist[0]/len(pos_df_dict[key]), name=key))
    #fig.add_trace(go.Histogram(x=tmp_data["chisq"], name=key))

fig.show()

405
